In [2]:
#!pip install --upgrade pip
!pip install pyathena

  Using cached pyathena-3.3.0-py3-none-any.whl.metadata (86 kB)
  Using cached urllib3-2.0.7-py3-none-any.whl.metadata (6.6 kB)
Using cached pyathena-3.3.0-py3-none-any.whl (80 kB)
Using cached urllib3-2.0.7-py3-none-any.whl (124 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.
sagemaker 2.199.0 requires urllib3<1.27, but you have urllib3 2.0.7 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
#Upgrade de la version de seaborn para poder hacer graficos mas facil 
!pip install seaborn==0.12.2

  Using cached seaborn-0.12.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.11.2
    Uninstalling seaborn-0.11.2:
      Successfully uninstalled seaborn-0.11.2

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
#Libraries
from pyathena import connect
import pandas as pd
import datetime as dt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime


In [5]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'
origin_name = 'AwsDataCatalog'
database_name = 'analytics'
tablaSource = 'source_fraud_fechas_label'


In [18]:
#Connection

conn = connect(s3_staging_dir=f'{bucket_name}queryresults/',
               region_name='us-east-1')
df_source = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{tablaSource} where try(cast(id_receiver as INTEGER)) is not null limit 100000;', conn)


/tmp/ipykernel_253/3076989322.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_source = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{tablaSource} where try(cast(id_receiver as INTEGER)) is not null limit 100000;', conn)


## Creo las nuevas variables
### sender_days_to_last_transaction_365 y sender_days_to_last_transaction_more365
### sender_minutes_since_last_transaction_2days
### net_amount_receiver?

In [20]:
#defino corte en funcion deñ percenti
df_source['sender_days_to_last_transaction'].describe()

count    100000.000000
mean        208.605490
std         192.434166
min           0.000000
25%          44.000000
50%         148.000000
75%         337.000000
max         725.000000
Name: sender_days_to_last_transaction, dtype: float64

In [21]:
#defino dos nuevas variables a partir de la anterior
df_source['sender_days_to_last_transaction_365'] = df_source['sender_days_to_last_transaction']
df_source['sender_days_to_last_transaction_more365'] = 0
#corto en un maximo de dias y el resto lo pongo enlas variables nuevas
df_source.sender_days_to_last_transaction_more365[df_source['sender_days_to_last_transaction']> 365] = 1
df_source.sender_days_to_last_transaction_365[df_source['sender_days_to_last_transaction']> 365] = 0 

/tmp/ipykernel_253/461507757.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_source.sender_days_to_last_transaction_more365[df_source['sender_days_to_last_transaction']> 365] = 1
/tmp/ipykernel_253/461507757.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_source.sender_days_to_last_transaction_365[df_source['sender_days_to_last_transaction']> 365] = 0


In [22]:
#defino dos nuevas variables a partir de la anterior
df_source['sender_minutes_since_last_transaction_2days'] = df_source['sender_minutes_since_last_transaction']
#corto en un maximo de dias y el resto lo pongo enlas variables nuevas
df_source.sender_minutes_since_last_transaction_2days[df_source['sender_minutes_since_last_transaction']> 2*24*60] = 0

/tmp/ipykernel_253/3931662022.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_source.sender_minutes_since_last_transaction_2days[df_source['sender_minutes_since_last_transaction']> 2*24*60] = 0


## Estandarizo variables
### sender_days_to_last_transaction_365, sender_minutes_since_last_transaction_2days, net_amount_receiver, sender_sending_days, branch_working_days, receiver_transaction_count

In [35]:
df_source['receiver_transaction_count'].describe()

count    100000.000000
mean         16.401880
std          22.274392
min           1.000000
25%           2.000000
50%           8.000000
75%          21.000000
max         500.000000
Name: receiver_transaction_count, dtype: float64

In [36]:
df_source['sender_sending_days'].describe()

count    100000.00000
mean        255.57263
std         209.18795
min           0.00000
25%          69.00000
50%         211.00000
75%         418.00000
max         728.00000
Name: sender_sending_days, dtype: float64

In [37]:
df_source['branch_working_days'].describe()

count    100000.000000
mean       1581.962730
std        1390.620895
min          -3.000000
25%         564.000000
50%        1165.000000
75%        2207.000000
max        6892.000000
Name: branch_working_days, dtype: float64

In [38]:
df_source['net_amount_receiver'].describe()

count    100000.000000
mean        464.277075
std         576.411078
min           1.000000
25%         101.000000
50%         240.000000
75%         590.000000
max        7992.000000
Name: net_amount_receiver, dtype: float64

In [40]:
df_source['sender_minutes_since_last_transaction_2days'].describe()

count    100000.000000
mean         98.863860
std         432.330646
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max        2880.000000
Name: sender_minutes_since_last_transaction_2days, dtype: float64

In [41]:
df_source['sender_days_to_last_transaction_365'].describe()

count    100000.000000
mean         93.523330
std         107.203622
min           0.000000
25%           0.000000
50%          52.000000
75%         163.000000
max         365.000000
Name: sender_days_to_last_transaction_365, dtype: float64